In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import os
import numpy as np
import math
import calendar

* Q1 - rebalance portfolio 
* Q2 - define period for which we take into account the size (Market Cap) - moving average of the size over the period Rebalancing every month
* Q3 - Add rebalancing every REB months e.g. every 12 months, so you keep the same portfolio composition for 12 months each time.
* Q4 - change percentage of companies taken to each of 2 portfolios

In [35]:

def pull_stock_data(tickers:str, start:str, end:str, interval:str, stripdateindex:str, column:str) -> pd.DataFrame:
    """
    Returns a dataframe with chosen stock information for last day of the month.

    Parameters:
    -------------
    tickers (str): Yahoo finance tickers for companies divided ONLY by single space
    start (str): start date of the period (format yyyy-mm-dd)
    end (str): end date of the period (format yyyy-mm-dd)
    interval (str): wanted interval (1d, 1m, 1y)
    stripdateindex (str): argument for stripping datetime index down (d - day, m - month, y- year)
    columns (list): list of wanted values, args same as in yahoo finance
    """
    # data download
    df = yf.download(tickers = tickers, start = start, end = end, interval = interval, groupby = 'ticker')
    # changing index from datetime to just year and month
    df['Date'] = df.index
    dfg = df.groupby([df.index.year, df.index.month], as_index=False).last()
    dfg.reset_index(inplace=True, drop=True)
    dfg.set_index('Date', inplace=True)
    dfg.index = pd.to_datetime(dfg.index).to_period(stripdateindex)
    # dropping na rows
    dfg.dropna(inplace = True)
    return dfg[column]

    
def save_to_desktop(dataframe:pd.DataFrame, file_name:str):
    """
    Saves dataframe to desktop in csv format under filename provided

    Parameters:
    ------------
    dataframe (pd.DataFrame): dataframe that is to be saved
    file_name (str): name of the file
    """
    # defining path to desktop on running unit
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    # saving file under chosen name on desktop
    dataframe.to_csv(desktop+'/'+file_name+'.csv', sep=';')


In [69]:
tick = "NKE MSFT XOM INTC CAT WMT JPM F UPS MKC" # stocks universe
st = '2000-12-01'
en = '2020-12-31'
intv = '1d'
strpdt = 'm'
col = ['Close', 'Volume']

stocks_data = pull_stock_data(tick, st, en, intv, strpdt, col)
stocks_data.info()

[*********************100%***********************]  10 of 10 completed
<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 241 entries, 2000-12 to 2020-12
Freq: M
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, CAT)    241 non-null    float64
 1   (Close, F)      241 non-null    float64
 2   (Close, INTC)   241 non-null    float64
 3   (Close, JPM)    241 non-null    float64
 4   (Close, MKC)    241 non-null    float64
 5   (Close, MSFT)   241 non-null    float64
 6   (Close, NKE)    241 non-null    float64
 7   (Close, UPS)    241 non-null    float64
 8   (Close, WMT)    241 non-null    float64
 9   (Close, XOM)    241 non-null    float64
 10  (Volume, CAT)   241 non-null    int64  
 11  (Volume, F)     241 non-null    int64  
 12  (Volume, INTC)  241 non-null    int64  
 13  (Volume, JPM)   241 non-null    int64  
 14  (Volume, MKC)   241 non-null    int64  
 15  (Volume, MSFT)  241 non-null    int

/opt/miniconda3/envs/time-series-env/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


calculating the stock returns

In [70]:
# we will not need Volume - will use Market Cap instead
stocks_data = stocks_data.drop(stocks_data.filter(regex='Volume').columns, axis=1)
stocks_data['Close'] = stocks_data['Close'].pct_change()

### Market Capitalization
* data retrieved from Bloomberg with daily frequency - need to resample it to 1 month (last day of each month) and join to data frame

In [71]:
mkt_cap = pd.read_excel("mkt_cap_G2.xlsx", parse_dates = True) #we have it in easy format so pandas will handle date parsing
# rename columns to match the ones in returns data frame - luckily Bloomberg tickers are the same, just with "US EQUITY" added
mkt_cap = mkt_cap.rename(columns={c : c.partition(" ")[0] for c in mkt_cap.columns})
# Get data, only for the last day of each month (and year ofc)
mth_mkt_cap = mkt_cap.groupby([mkt_cap['Date'].dt.year, mkt_cap['Date'].dt.month], as_index=False).last().set_index("Date")
# make index have the same format as in stocks_data
mth_mkt_cap.index = mth_mkt_cap.index.to_period("m")
# create MultiIndex for Market Cap and each company
mth_mkt_cap.columns = pd.MultiIndex.from_product([["MktCap"], mth_mkt_cap.columns])
# join MktCap to stocks_data
stocks_data = stocks_data.join(mth_mkt_cap, on=["Date"])
stocks_data.head()

Close                                                              \
              CAT         F      INTC       JPM       MKC      MSFT       NKE   
Date                                                                            
2000-12       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2001-01 -0.065363  0.202773  0.230769  0.210234  0.010745  0.407781 -0.014379   
2001-02 -0.059249 -0.013480 -0.228041 -0.151482  0.078189 -0.033777 -0.290493   
2001-03  0.066827  0.011147 -0.078775 -0.037720  0.068448 -0.073093  0.033820   
2001-04  0.131140  0.048364  0.174727  0.068597 -0.064063  0.238857  0.036183   

                                           MktCap                          \
              UPS       WMT       XOM           F         CAT        INTC   
Date                                                                        
2000-12       NaN       NaN       NaN  44443.0234  16263.8134  202320.625   
2001-01  0.053617  0.069176 -0.032063  53454.8834  15200.7572  249010.000   
2001-02 -0.086753 -0.118134 -0.036839  52734.3138  14300.1244  192225.625   
2001-03  0.006545  0.008185 -0.000617  51601.6286  15239.9495  176767.375   
2001-04  0.009666  0.024554  0.093827  56037.1191  17233.6594  207653.380   

                                                                      \
                 JPM        MKC         MSFT         NKE         UPS   
Date                                                                   
2000-12   59539.5761  2391.9712  231290.1560  15072.8352  66471.9433   
2001-01  106048.2150  2423.6188  325463.1250  14836.3903  70035.9709   
2001-02   89983.8100  2586.4043  314788.0469  10528.0554  61257.7120   
2001-03   88625.5953  2754.3177  291779.1748  10932.4622  63622.2866   
2001-04   94705.0348  2611.0851  361472.7148  11338.7800  64957.4753   

                                   
                 WMT          XOM  
Date                               
2000-12  237291.5543  302194.7500  
2001-01  253706.5465  292505.4000  
2001-02  223735.2518  281729.8000  
2001-03  225566.5845  279888.1436  
2001-04  231301.7500  306149.2533

### Not necessary
calculating stock excess return - *not sure if that's actually needed*

In [5]:
fama = pd.read_csv(r"F-F_Research_Data_Factors.CSV", sep=';')
fama['Date'] = pd.to_datetime(fama['Date'].astype('string'), format='%Y%m')
fama.set_index('Date', inplace=True)
fama.index = fama.index.to_period('m')
# Values given in CSV are plain percentages - division by 100
fama = fama.divide(100)
fama.head()


,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031


In [6]:
# Join risk free rates to stock returns in periods
# dffull = pd.merge(stocks_data,fama, how='left', left_index=True, right_index=True)
# # drop first NULL row which is causing regression to fail
# dffull = dffull.dropna()

In [7]:
# Calculate excess stock return for each company
# for i in dffull.columns.tolist()[:10]:
#     dffull[i+'_re'] = dffull[i]-dffull['RF']
# # Calculate excess Market return
# dffull['reM'] = dffull['Mkt-RF']-dffull['RF']
# dffull.info()

### Portfolio building functions

periods (PS) wyznacza od kiedy robimy rebalancing (bo musimy zebrać dane pod to)
Piersze PS periodów powinno być widoczne w DataFrame ale mieć NaN (aby shape się zgadzał)

In [73]:
# reshape DataFrame
c = stocks_data['Close'].melt(ignore_index=False, value_name="return", var_name='company') #.reset_index()
s = stocks_data['MktCap'].melt(ignore_index=False, value_name="mkt_cap", var_name='company')
df = pd.merge(c, s, on=["Date",'company'])
df

,company,return,mkt_cap
Date,,,
2000-12,CAT,NaN,16263.8134
2001-01,CAT,-0.065363,15200.7572
2001-02,CAT,-0.059249,14300.1244
2001-03,CAT,0.066827,15239.9495
2001-04,CAT,0.131140,17233.6594
...,...,...,...
2020-08,XOM,-0.050856,168875.6375
2020-09,XOM,-0.140461,145155.2488
2020-10,XOM,-0.049811,137924.9699


In [76]:
def rebalance(df_slice, sort_var, p1, p2):
    """
    Rebalancing returns a list of 2 (for now) portfolios based on the value of decision variable in period taken into account. 
    Period is defined by the length of df_slice.
    p1, p2 - percentages given to each class of companies, they are in ASCENDING order, so the higher portfolio number the bigger the sort_var value
    TODO: Adjust to return "n" portfolios - this would require changing to sth more sophisticated than head/tail and adjusting form in which percentages are given
    """
    # 1st part - decide on size of each portoflio - based on universe size and percentages assigned to each class of stocks (p1, p2)
    universe_size = df_slice['company'].nunique()
    small_cnt = int((universe_size*p1)//1)
    big_cnt = round(universe_size*p2)

    period_avg = df_slice.groupby('company').mean().reset_index()
    # SORTING by variable - biggest values are in the BOTTOM of dataframe
    temp_sort = period_avg.sort_values(sort_var, ascending = True)
    # probably some loop over every percentage to get proper companies from sorted list
    # we need to return only the names of companies, because the values of returns at this stage are already aggregated for the whole `period`
    small_comp = temp_sort.head(small_cnt)['company'].values
    big_comp = temp_sort.tail(big_cnt)['company'].values

    return [small_comp, big_comp]


def get_portfolio_stats(date, portfolios, df, sort_var):
    """
    portfolios - list of lists of company tickers. The portfolios are increasing order with respect to decision variable value. (P2 will have greater `size` than P1)
    We are interested in average size (mkt_cap), average return and composition of each portfolio
    Function iterates over list of portfolios and calculates metrics for each of them assuming they are equally weighted
    Returns dict of statistics with portfolio number which is used later to construct pd.DataFrame
    """
    ptfs = {}
    for idx, p in enumerate(portfolios, 1):
        p_period_df = df.loc[(df.index == date) & (df.company.isin(p))]
        
        p_name = "ptf_" + str(idx)
        ptfs[p_name+"_return"] = p_period_df['return'].mean()
        ptfs[p_name+"_"+sort_var] = p_period_df[sort_var].mean()
        ptfs[p_name+"_stocks"] = p

    return ptfs

def constrcut_portfolios(df, decision_var, reb, per, small_p, big_p):
    """
    Function that simulates the process of constructing portfolios according to decision variable. 
    It assumes that each period is the end of month for which data is available (so at time "T" we already now the values of decision_var at period "T")
    df - DataFrame in long format with stocks under "company" column
    reb - rebalancing period, how often we change composition of portfolios
    per - defines how many periods are taken into account when rebalancing. First rebalance happens after per periods past (to gather data)
    small_p - percentage of stocks considered as "small" or "big" based on decision variables (low/high) would be more generalized naming
    """
    portfolios = []
    test_l = []
    first_reb = True
    reb_counter = 0
    
    for p_idx, d in enumerate(df.index.unique(), 1):
        reb_counter += 1
        
        if (not first_reb and reb_counter%reb == 0) or (first_reb and p_idx==per):
            # define slice based on periods taken into account for rebalancing decision (rebalancing could be every 12 months, but periods considered could be e.g. 6 months)
            # print(reb_counter, "rebalance", d)
            slice = df[(df.index >= d - per+1) & (df.index <= d)]
            # return portfolios after rebalancing
            portfolios = rebalance(slice, decision_var, small_p, big_p)

            # a couple of settings after first rebalancing
            if first_reb and p_idx==per:
                first_reb = False
                reb_counter = 0 #start counting until next rebalancing from zero 
        
        # this is done for each period 
        # calculation of metrics
        stats = get_portfolio_stats(d, portfolios, df, decision_var)
        test_l.append(stats)
    
    return pd.DataFrame(test_l, index=df.index.unique())


QUESTION 1 - portfolios sorted based on the past size of company.
p1 - 50% smallest (by mkt_cap) stocks in t-1,
p2 - 50% biggest (by mkt_cap) stocks in t-1

In [77]:
Q1 = constrcut_portfolios(df=df, decision_var="mkt_cap", reb=1, per=1, small_p=.5, big_p=.5)
Q1

,ptf_1_return,ptf_1_mkt_cap,ptf_1_stocks,ptf_2_return,ptf_2_mkt_cap,ptf_2_stocks
Date,,,,,,
2000-12,NaN,27542.24386,"[MKC, NKE, CAT, F, JPM]",NaN,207913.80572,"[UPS, INTC, MSFT, WMT, XOM]"
2001-01,0.037479,31190.32412,"[MKC, NKE, CAT, F, UPS]",0.177179,245346.65730,"[JPM, INTC, WMT, XOM, MSFT]"
2001-02,-0.074357,28281.32198,"[MKC, NKE, CAT, F, UPS]",-0.113654,220492.50674,"[JPM, INTC, WMT, XOM, MSFT]"
2001-03,0.037357,28830.12892,"[MKC, NKE, CAT, F, UPS]",-0.036404,212525.37464,"[JPM, INTC, WMT, XOM, MSFT]"
2001-04,0.032258,30435.62378,"[MKC, NKE, CAT, F, UPS]",0.120112,240256.42658,"[JPM, INTC, WMT, XOM, MSFT]"
...,...,...,...,...,...,...
2020-08,0.051196,88357.68934,"[F, MKC, CAT, UPS, XOM]",0.084723,559303.05524,"[NKE, INTC, JPM, WMT, MSFT]"
2020-09,-0.031228,84437.50482,"[MKC, F, CAT, UPS, XOM]",0.007879,539523.28956,"[NKE, INTC, JPM, WMT, MSFT]"
2020-10,0.007334,82686.58252,"[MKC, F, CAT, UPS, XOM]",-0.043122,518541.63858,"[INTC, NKE, JPM, WMT, MSFT]"


QUESTION 2 - portfolios sorted based on the average past size of company in previous n periods.
p1 - 50% smallest (by mkt_cap) stocks in t-1,
p2 - 50% biggest (by mkt_cap) stocks in t-1

In [78]:
# in thissetting we don't construct portfolios until 6 months of data is available 
# then rebalance them every month, but considering 6 period rolling average size
Q2 = constrcut_portfolios(df=df, decision_var="mkt_cap", reb=1, per=6, small_p=.5, big_p=.5)
Q2

,ptf_1_return,ptf_1_mkt_cap,ptf_1_stocks,ptf_2_return,ptf_2_mkt_cap,ptf_2_stocks
Date,,,,,,
2000-12,NaN,NaN,NaN,NaN,NaN,NaN
2001-01,NaN,NaN,NaN,NaN,NaN,NaN
2001-02,NaN,NaN,NaN,NaN,NaN,NaN
2001-03,NaN,NaN,NaN,NaN,NaN,NaN
2001-04,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-08,0.090626,89489.74778,"[F, MKC, CAT, UPS, NKE]",0.045293,558170.99680,"[XOM, INTC, JPM, WMT, MSFT]"
2020-09,0.021263,94572.13674,"[F, MKC, CAT, UPS, NKE]",-0.044612,529388.65764,"[XOM, INTC, JPM, WMT, MSFT]"
2020-10,0.007334,82686.58252,"[MKC, F, CAT, UPS, XOM]",-0.043122,518541.63858,"[NKE, INTC, JPM, WMT, MSFT]"


QUESTION 3 - portfolios sorted based on the average past size of company in previous n periods, portfolio components change every REB periods.
p1 - 50% smallest (by size) stocks in t-1,
p2 - 50% biggest (by size) stocks in t-1

In [12]:
# left it here to spot difference between decision variables Volume vs. MktCap
Q3 = constrcut_portfolios(df=df, decision_var="volume", reb=12, per=6, small_p=.5, big_p=.5)
Q3

,ptf_1_return,ptf_1_volume,ptf_1_stocks,ptf_2_return,ptf_2_volume,ptf_2_stocks
Date,,,,,,
2000-12,NaN,NaN,NaN,NaN,NaN,NaN
2001-01,NaN,NaN,NaN,NaN,NaN,NaN
2001-02,NaN,NaN,NaN,NaN,NaN,NaN
2001-03,NaN,NaN,NaN,NaN,NaN,NaN
2001-04,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-08,0.098878,5241040.0,"[MKC, CAT, UPS, NKE, WMT]",0.037041,32891360.0,"[JPM, INTC, XOM, MSFT, F]"
2020-09,0.027482,6247880.0,"[MKC, CAT, UPS, NKE, WMT]",-0.050831,31221860.0,"[JPM, INTC, XOM, MSFT, F]"
2020-10,-0.025193,4996360.0,"[MKC, CAT, UPS, NKE, WMT]",-0.010596,45232160.0,"[JPM, INTC, XOM, MSFT, F]"


In [79]:
Q3 = constrcut_portfolios(df=df, decision_var="mkt_cap", reb=12, per=6, small_p=.5, big_p=.5)
Q3

,ptf_1_return,ptf_1_mkt_cap,ptf_1_stocks,ptf_2_return,ptf_2_mkt_cap,ptf_2_stocks
Date,,,,,,
2000-12,NaN,NaN,NaN,NaN,NaN,NaN
2001-01,NaN,NaN,NaN,NaN,NaN,NaN
2001-02,NaN,NaN,NaN,NaN,NaN,NaN
2001-03,NaN,NaN,NaN,NaN,NaN,NaN
2001-04,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-08,0.090626,89489.74778,"[MKC, F, CAT, UPS, NKE]",0.045293,558170.99680,"[XOM, INTC, WMT, JPM, MSFT]"
2020-09,0.021263,94572.13674,"[MKC, F, CAT, UPS, NKE]",-0.044612,529388.65764,"[XOM, INTC, WMT, JPM, MSFT]"
2020-10,0.008597,92802.54256,"[MKC, F, CAT, UPS, NKE]",-0.044386,508425.67854,"[XOM, INTC, WMT, JPM, MSFT]"


QUESTION 4 - portfolios sorted based on the average past size of company in previous n periods, portfolio components change every REB periods.
p1 - 30% smallest (by size) stocks in t-1,
p2 - 30% biggest (by size) stocks in t-1

In [80]:
Q4 = constrcut_portfolios(df=df, decision_var="mkt_cap", reb=12, per=6, small_p=.3, big_p=.3)
Q4

,ptf_1_return,ptf_1_mkt_cap,ptf_1_stocks,ptf_2_return,ptf_2_mkt_cap,ptf_2_stocks
Date,,,,,,
2000-12,NaN,NaN,NaN,NaN,NaN,NaN
2001-01,NaN,NaN,NaN,NaN,NaN,NaN
2001-02,NaN,NaN,NaN,NaN,NaN,NaN
2001-03,NaN,NaN,NaN,NaN,NaN,NaN
2001-04,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-08,0.053572,43882.437500,"[MKC, F, CAT]",0.069952,801762.998833,"[WMT, JPM, MSFT]"
2020-09,-0.011359,44389.339467,"[MKC, F, CAT]",-0.032963,760522.566467,"[WMT, JPM, MSFT]"
2020-10,0.047871,46621.006800,"[MKC, F, CAT]",-0.009092,740914.660933,"[WMT, JPM, MSFT]"


___

In [14]:
new_file = pd.read_excel('/Users/stefano/Downloads/investment-theory-main/mkt_cap_G2.xlsx')

In [15]:
last_days=[]
years=range(2000,2021)
months=range(1,13)

In [16]:
for y in years:
    for x in months:
        last_days.append([y, x, calendar.monthrange(y, x)[1]])

In [17]:
last_days_df = pd.DataFrame(data = last_days, columns=['year', 'month', 'day'])

In [18]:
last_days_df['Date']=last_days_df['year'].astype(str) + '-' + last_days_df['month'].astype(str) + '-' + last_days_df['day'].astype(str)

In [19]:
last_days_df['Date']=pd.to_datetime(last_days_df['Date'], infer_datetime_format=True, errors='coerce')

In [20]:
last_days_df=last_days_df[last_days_df['Date']>'2000-11-30']

In [21]:
new_file=new_file[new_file['Date'].isin(last_days_df['Date'])==True]

In [22]:
new_file.reset_index(inplace=True, drop=True)

In [23]:
print(new_file.dtypes)

Date              datetime64[ns]
F US Equity              float64
CAT US Equity            float64
INTC US Equity           float64
JPM US Equity            float64
MKC US Equity            float64
MSFT US Equity           float64
NKE US Equity            float64
UPS US Equity            float64
WMT US Equity            float64
XOM US Equity            float64
dtype: object


In [24]:
assert stocks_data.shape[0] == new_file.shape[0]

In [25]:
stocks_data.reset_index(inplace=True, drop=True)

In [26]:
stocks_data = stocks_data.drop(stocks_data.filter(regex='Volume').columns, axis=1)

In [27]:
final_df= pd.concat([stocks_data, new_file], axis=1)

In [28]:
final_df.head()

,"(Close, CAT)","(Close, F)","(Close, INTC)","(Close, JPM)","(Close, MKC)","(Close, MSFT)","(Close, NKE)","(Close, UPS)","(Close, WMT)","(Close, XOM)",...,F US Equity,CAT US Equity,INTC US Equity,JPM US Equity,MKC US Equity,MSFT US Equity,NKE US Equity,UPS US Equity,WMT US Equity,XOM US Equity
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44443.0234,16263.8134,202320.625,59539.5761,2391.9712,231290.1560,15072.8352,66471.9433,237291.5543,302194.7500
1,-0.065363,0.202773,0.230769,0.210234,0.010745,0.407781,-0.014379,0.053617,0.069176,-0.032063,...,53454.8834,15200.7572,249010.000,106048.2150,2423.6188,325463.1250,14836.3903,70035.9709,253706.5465,292505.4000
2,-0.059249,-0.013480,-0.228041,-0.151482,0.078189,-0.033777,-0.290493,-0.086753,-0.118134,-0.036839,...,52734.3138,14300.1244,192225.625,89983.8100,2586.4043,314788.0469,10528.0554,61257.7120,223735.2518,281729.8000
3,0.066827,0.011147,-0.078775,-0.037720,0.068448,-0.073093,0.033820,0.006545,0.008185,-0.000617,...,51601.6286,15239.9495,176767.375,88625.5953,2754.3177,291779.1748,10932.4622,63622.2866,225566.5845,279888.1436
4,0.131140,0.048364,0.174727,0.068597,-0.064063,0.238857,0.036183,0.009666,0.024554,0.093827,...,56037.1191,17233.6594,207653.380,94705.0348,2611.0851,361472.7148,11338.7800,64957.4753,231301.7500,306149.2533


In [29]:
final_df.columns

Index([ ('Close', 'CAT'),    ('Close', 'F'), ('Close', 'INTC'),
        ('Close', 'JPM'),  ('Close', 'MKC'), ('Close', 'MSFT'),
        ('Close', 'NKE'),  ('Close', 'UPS'),  ('Close', 'WMT'),
        ('Close', 'XOM'),            'Date',     'F US Equity',
         'CAT US Equity',  'INTC US Equity',   'JPM US Equity',
         'MKC US Equity',  'MSFT US Equity',   'NKE US Equity',
         'UPS US Equity',   'WMT US Equity',   'XOM US Equity'],
      dtype='object')